GUID: 2147105s<br>
GitHub URL: https://github.com/markstrathie/2023_AI_Arts_A

# Generating Shakespearean Text with Neural Networks


<div>
<img src="images/William_Shakespeare_by_John_Taylor.jpg" width="300" alt="Portrait of William Shakespeare"/>
</div>

Welcome to my notebook about machine learning (ML) in the arts and humanities for beginners! This will show how a simple model can learn how to generate Shakespearean text. By the end of this notebook you will be able to input a prompt and receive back an original sample of Shakespearean prose.

While artifical intellegence (AI) attempts to use machines to mimic the problem-solving and decision-making capabilities of the human mind, machine learning focuses specifically on using algorithms that can generalise tasks, allowing the perfomance of tasks without exact instructions.

Machine learning is valuable skill to learn. In the arts and humanities, ML models have many creative use cases such as predicting gaps in historical texts, aiding in creative writing, or analysing language patterns.

While this might sound daunting, don't worry! We will go steadily through the high-level ML pipeline:

1. Framing the Problem
2. Getting the Data
3. Exploring the Data
4. Preparing the Data
5. Building and Training the Model
6. Evaluation and Interpretation of the Results

As you work through the notebook and come across sections of code, please ensure you click on each section and click <b>'Run'</b> at the top of the page. As code after is dependant on the code the comes before, this will ensure there are no errors. Additional to the full explainations of what the code does, you can see helpful comments next to various lines of code after the <b>'#'</b> symbol. Have fun!

<i>Image from https://upload.wikimedia.org/wikipedia/commons/5/5e/William_Shakespeare_by_John_Taylor.jpg</i>

# Framing the Problem

Framing a problem involves defining the ideal outcome, identifying the model's output, and defining success metrics. This is important to ensure we even need to use ML in the first place. It then defines how we carry out the rest of the steps along the pipeline.

In our case, given a sequence of characters from a dataset of Shakespeare's scripts, we want to train a model to predict the next character in the sequence. Longer sequences of text can then be generated by calling the model repeatedly. More speficially, we need the output to be in English, formatted correctly with correct grammer and punctuation. Success can easily be measured by reading the output in comparison to Shakespeare's real scripts.

In a normal machine learning problem, various model types would be considered each with their own advantages and disadvantages. However, in this notebook we have already chosen to use a <b>Neural Network</b>. This is because it is well-suited to work with nonlinear data with large number of inputs, such as our scripts.

Google has a well written guide to framing Machine learning problems if you would like to learn more - https://developers.google.com/machine-learning/problem-framing/ml-framing

### What is a Neural Network?

A neural network is a computer model inspired by the structure of the human brain and its interconnected network of neurons.

<div>
<img src="images/800px-Neural_network_example.svg.png" width="300" alt="Diagram of a neural network"/>
</div>

- Input Layer: receives the initial data and passes them to the next layer.

- Hidden Layers: intermediary layers that consist of multiple neurons that perform computations on the data.

- Output Layer: the final layer that produces the network's output based on the information processed through the hidden layers.

Neural networks are trained using a process called supervised learning, where they learn from labeled training data. The learning involves adjusting the weights associated with each connection between neurons to minimise the difference between predicted outputs and actual target values. This optimisation process enables the network to make predictions on unseen data.

Different types of neural networks exist, but we will only use a <b>Recurrent Neural Network (RNN)</b>. These are designed to handle sequential data by incorporating feedback loops, making them suitable for tasks like natural language processing - perfect for our text generation!

To acheive our goal we will be using <b>TensorFlow</b> which is a machine learning framework developed by Google Brain. It allows us to more easily develop, train, and deploy machine learning models, particularly neural networks. This notebook will be closely based on TensorFlow's tutorial located here - https://www.tensorflow.org/text/tutorials/text_generation.

For further information on working with RNNs - http://karpathy.github.io/2015/05/21/rnn-effectiveness/

<i>Diagram from https://en.wikipedia.org/wiki/Neural_network#/media/File:Neural_network_example.svg</i>

# Getting the Data

In order to create a working machine learning model, we first need data to train and test it. In general, neural networks perform best when utilising large datasets. The code below downloads over 1 million characters from the works of Shakespeare. This is then stored locally in the variable <b>shakespeare_text</b>. This text can then be accessed for further processing within our notebook.

In [ ]:
import tensorflow as tf # import our machine learning framework so we can use it

shakespeare_url = "https://homl.info/shakespeare"  # shortcut URL to access our shakespeare text
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url) # creates file to save the text
with open(filepath) as f: # opens the file
    shakespeare_text = f.read() # reads in the file to a variable that we can call later on

# Exploring the Data

Let's take a look at the text we downloaded. This helps us understand what processing needs undertaken before we can use it for training. In this case, it also gives us an idea of what our ideal output data should look like. We can view a section of the data by indexing and slicing it. The code <b>[:250]</b> means slicing off the rest of the data after 250 characters. The notation is <b>data[start:stop]</b>. We will see slicing being used later on to make subsets of the data.

In [ ]:
print(shakespeare_text[:250]) # not relevant to machine learning but relevant to exploring the data - displays the first 250 characters

Notice that the data is made up of English words, arranged as a script with speakers being identified before each line. It also contains punctuation and correct grammer. This is the format we are trying to reproduce, ultimately generating original Shakespearean lines.

# Preparing the Data

Data usually needs to be processed before our model can be trained on it. This can involve making sure everything is labelled, the format is consistant, or there are no missing or invalid values. Additonally, before training our RNN with this data, we need to convert the strings to a numerical representation so the weights connecting the neurons can be calculated. This is called <b>vectorizing the text</b>.

The code below sets up a TextVectorization layer to convert our Shakspearien text characters into numerical representations. See if you can spot the  printed output that includes a list of numbers representing the characters in the text, and the total number of characters.

In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character",
                                                   standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [ ]:
print(text_vec_layer([shakespeare_text]))

The code below now modifies the numerical representation by removing tokens (characters) for padding (whitespace) and unknown characters, calculates the count of distinct characters in the text, and determines the total number of characters in the processed text data. These values are important for further use of our dataset.

In [ ]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

Here we output the tokens and dataset size:

In [ ]:
print(n_tokens, dataset_size)

Below we use code to define a function called <b>to_dataset</b>. This function is designed to prepare a TensorFlow dataset suitable for training our RNN model. It converts an inputted sequence into input-target pairs structured as batches, ready for training.

In [ ]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

The code below now uses the function on our data to partition sequences of it into three subsets: training, validation, and test sets. It creates sequences of a specified length, allowing for the proper formatting of the data into our desired input-target pairs. The training set is then <b>shuffled</b>, while validation and test sets are kept in their respective order without shuffling.

Shuffling is an essential step during the training phase of a machine learning model:

- Reduces Bias: Shuffling helps prevent the model from learning patterns or sequences present in the original order of the dataset.

- Reduces Overfitting: Shuffling can aid in reducing overfitting by ensuring that the model does not memorise patterns specific to the order of the data but instead learns more generalised features.

See if you can spot the subset sizes in the code below being created using slicing:

- Training: 1,000,000 characters
- Validation: 60,000 characters
- Test: 55,394 characters

Notice the number 42 as the <b>tf.random.set_seed</b> value. This is to ensure reproducibility and could be any value. However, it is standard practise to use the number 42. This is a reference to <i>The Hitchhiker's Guide to the Galaxy</i>, where the supercomputer Deep Thought reveals that the answer to the great question of “life, the universe and everything” is 42.

In [ ]:
length = 100
tf.random.set_seed(42) # the answer to the great question of “life, the universe and everything”
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

# Building and Training the Model

This is where the magic happens! This code sets up and trains our simple RNN model on our processed subsets of our dataset, monitoring the accuracy and saving the best-performing model for later use. Our RNN has 3 layers - <b>Embedding</b>, <b>GRU</b>, and <b>Dense</b>.

Part of the setup involves configuring a <b>loss function</b>, <b>optimiser</b>, and <b>checkpoints</b>. If you are interested in more details you can follow the links to the official TensorFlow Documentation.

- A loss function measures our model's performance by quantifying the difference between predicted values and actual target values. We are using sparse categorical crossentropy because it's well-suited for dealing with a large number of discrete output classes (like each character in a vocabulary) - read more at https://www.tensorflow.org/api_docs/python/tf/keras/metrics/sparse_categorical_crossentropy
- An optimiser is a method used to update our model's parameters (weights and biases) iteratively during the training phase to minimise the loss function. We are using Nadam because it provides more stable training in tasks with varied data characteristics, common in text data - read more at https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/Nadam
- Checkpoints are to help with evaluation at various points of the training, so we can pick the best performing stage of our model - read more at https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint

When you run the code below it will show the process of each <b>Epoch</b>, which refers to one complete pass through the entire dataset. Training for too few epochs might result in underfitting (inadequate learning), while training for too many epochs could lead to overfitting (memorisation of training data). We are going to train for 10 epochs which we hope will allow for a a good balance.

You will also eventually see each epoch's <b>loss</b> and <b>accuracy</b>. Loss show's the output of the loss function and accuracy is a percentage of correctly predicted examples out of the total. Obviously the closest to 1 accuracy is the best outcome we can hope for (100% accurate).

You may see a warning on certain epochs reading 'WARNING:absl:Found untraced functions such as gru_cell_1_layer_call_fn, gru_cell_1_layer_call_and_return_conditional_losses while saving (showing 2 of 2). These functions will not be directly callable after loading.' This is not a concern as we do not intent to reload our best model in the future, so we can safely ignore this warning. We can still use our best model in our current local environment after training - read more at https://stackoverflow.com/questions/65697623/tensorflow-warning-found-untraced-functions-such-as-lstm-cell-6-layer-call-and

### Training a RNN is hardware intensive and will take a while, up to many hours, so please be patient...

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU by chosing same value as above code (42)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16), # layer 1
    tf.keras.layers.GRU(128, return_sequences=True), # layer 2
    tf.keras.layers.Dense(n_tokens, activation="softmax") # layer 3
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", # define loss function and optimiser
              metrics=["accuracy"])
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True) # configure checkpoints
history = model.fit(train_set, validation_data=valid_set, epochs=10, # selecting to train for 10 epochs
                    callbacks=[model_ckpt])

The <b>tf.keras.Sequential</b> function below creates a new model by combining it with another 2 layers:

1. Text vectorizating as discussed above, so the characters are first converting into numerical data.
2. This allows for the Lambda Layer to subtract 2, removing the padding and unknown tokens as discussed above.

The output from these steps in now fed in to the trained model from above.

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer, # converts text to numerical data
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model # trained model from above
])

### Testing and refining our model

The next section of code is more complex. While it may be hard to understand each specific line of code, it is useful to help understand the general process.

We now use our <b>shakespeare_model</b> to predict the next character following the input sequence "To be or not to b". It computes the most probable character ID, then maps this ID back to the actual character using the vocabulary obtained from the <b>text_vec_layer</b>. We are obviously expecting the output to be 'e'.

In [ ]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)  # choose the most probable character ID
text_vec_layer.get_vocabulary()[y_pred + 2]

So far so good. This next code is more complex but involves random sampling based on probability distributions (log probabilities 50%, 40%, and 10%). It is used to generate random data points that follow a specified distribution. Each of the 8 samples correspond to selecting the next word based on the predicted probability distribution over the vocabulary. This code helps contribute to the diversity and randomness of generated text.

In [ ]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)  # draw 8 samples

The below code defines the function <b>next_char</b>. It utilises our shakespeare_model to predict the probabilities for the next character in our text sequence. It then generates the next character based on these probabilities and the temperature parameter. <b>'Temperature'</b> allows for controlled randomness in each character selection.

In [ ]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

Next we create an <b>extend_text</b> function that calls our <b>next_char</b> function iteratively to predict and add characters to the sequence, up until specified number of characters (<b>n_chars</b>). Again, the <b>temperature</b> parameter is used for controlling randomness.

*Adjust n_chars=250 to change output size in next section

In [ ]:
def extend_text(text, n_chars=250, temperature=1):
    for _ in range(n_chars): # for loop to create n number of characters
        text += next_char(text, temperature) # append new character to end of current text
    return text # produce an output when the function is called

Here again we use 42 for reproducibility by ensuring that the same sequence of random numbers is generated each time the code is executed.

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

# Evaluation and Interpretation of the Results

Hurray! You finished your model. Here you can play around with it. Below are 3 code cells with different randomness levels for the text generation and different input prompts. Have a go at adjusting them and seeing the results. Again, be patient while the model runs.

Some points to consider in your experiments:

- Try various prompts by adjust the text in the quotes
- A temperature of 0.01 is as non-random as possible and produces the same output every time
- 1 is the base value the model has
- 100 is extremely random
- The output size is set to 250 characters in the extend_text function above*
- While increasing this will produce a larger output, it will take the model longer to produce, so bear this in mind
- Have fun!


In [ ]:
print(extend_text("Q", temperature=0.01))

In [ ]:
print(extend_text("You sir are an egg!", temperature=1))

In [ ]:
print(extend_text("To be or not to be", temperature=100))

# Critical Reflection

To evaluate the result of my generative AI code I assessed it as discussed when framing the problem. I looked at the output in terms of coherence, grammer, format, and similarity to Shakespearean style. I also thought about the the diversity and novelty of generated sequences to consider if it just reproduces the same text repeatedly or produces varied outputs. I also demonstrated my results to my peers and we discussed how well we thought it performed.

Overall, my peers and I considered the AI code to be adequate. It was cleary capable of generating English words, basic sentences and punctuation. It was in the format of a script with speakers being announced. It uses Shakespearean style words such as duke and has a similar sentence structure. However, there are areas that fall short of a fully convincing model. It does not use capital letters at all. It creates words that do not exist. It uses incorrect grammer. The biggest problem is its comprehension. When adjusting for an output that is not too repetitive, it generates sentences that simply do not make sense. See the 250 character example below with a temperature of 1 and the prompt "You sir are an egg!":

    You sir are an egg! which i have bringing advised and my abjed.
    i have clamber man pay you?

    angelo:
    i trange your execution' this a freed it being a
    seeming so of froth. sir?

    lucentio:
    call mine own grave than love
    the prison of her duke? the new maids?
    duke herding 

People who had not had experience with AI text generation were immediately impressed with result, but on further inspection the realised its limitations. Others, including myself, noticed ways it could be improved. When training the model, it's greatest accuracy was only 60.7%. Nevertheless, we all found the 'temperature' parameter to be useful and enjoyed trying to find the balance between coherant but repetitive outputs, and random characters that did not even form words. A low temperature ensured all the words were spelled correctly, but each sentence was very repetitive. A high temperature just produced a collection of random symbols. If I was undertaking this task again I would work more on building and training a model with increased accuracy as I believe this to be the biggest limitation. 

I consider much of this code is useful and ready to apply to my own data. As mentioned above, the only major change I would make would be working on making the model more accurate after training, as I beleive this would improve the quality of the text generation it outputs. I can imagine applying it to a dataset made from historical documents, such as the Oxyrhynchus Papyri. This is a text collection made from thousands of pieces of parchment dating around 2000 years old. You can run the code below to watch the short video or read about the collection here - https://oxyrhynchus.web.ox.ac.uk/home

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('dEJ0hqbu904', width=800, height=450)

Due to their age, much of the text is in fragments the size of large cornflakes. The cultural heritage of these texts could be preserved by digitising, connecting, and augmenting them using generative AI. The current process is extremely time consuming and, using AI, it may be possibly to speed up the process. This is similar to the generative AI used successfully on the Dead Sea Scrolls which identified a second scribe - https://artsandculture.google.com/story/discover-the-secrets-of-the-dead-sea-scrolls/mAVBYIU6bF6wDA?hl=en

In addition to my code, other steps would need to be added to the ML pipeline before and after. To transform the content for the computer, the parchments would need to be scanned and stored digitally. Some sort of optical character recognition would need to be done on the images to separate the text data from the image data. From here, the process would be similar to what I carried out in this notebook - vectorize the text data by converting it into a numerical format suitable for the RNN model. Potentially, I would need to handle issues like missing data, encoding variations, and ensuring uniformity in format.

After the model is complete, I would need to pass the output down the pipline to another model to search the fragments against the generated outputs for a match. This would be an exciting process - to see human and AI collaboration being used creatively to connect each fragment like a giant jigsaw puzzle.

Throughout the entire machine learning process, there are several ethicic concerns I would need to be aware of:

- Bias and Misrepresentation: text generated could perpetuate biases or misrepresent historical or cultural texts. My data would need to be carefully curated and sanitised to mitigate this.

- Intellectual Property (IP): I would consider IP rights if generating content similar to existing texts, such as Shakespeare, ensuring proper attribution to avoid plagiarism.

- Impact on Society: there is potential to impact readers' perceptions if my AI-generated content is mistaken for authentic historical texts. I would need to promote transparency about my content's AI-generated nature to avoid misleading people.

- Privacy: aiming not to generate content that contains sensitive or private information that could harm individuals or communities.

Addressing these considerations is crucial in responsibly using generative AI, ensuring we can promote human-AI collaboration in a positive and creative way in the arts and humanities.